In [7]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key=os.getenv('GEMINI_KEY'))
embeddings= GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.getenv('GEMINI_KEY'))

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

system_prompt ="You are a helpful finance expert named fred in year 2025. First create a plan to get a answer from the research query. Then use the tools to get the answer  to the questions. Finally you use the answeres to each question to give the final verdict  "
tools = [TavilySearchResults(max_results=3)]
agent_executor = create_react_agent(llm , tools, state_modifier = system_prompt)

In [15]:
import operator
from pydantic import BaseModel, Field
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict

class PlanExecute(TypedDict):
    input:str
    plan:List[str]
    past_steps:Annotated[List[Tuple] , operator.add]
    response:str

class Plan(BaseModel):
    """Plan need to followed to be in future"""

    steps: List[str]= Field(
        description="different steps to be followed, should be in sorted order"
    )

In [16]:
from langchain.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_messages([
    (
        "system", """You are finnance research agent working in april 2025.\
            come up answers with simple step by step """
    ),
    ("placeholder","{messages}")   
])

In [17]:
planner = planner_prompt|llm.with_structured_output(Plan)
planner.invoke({"messages":[
    ("user", "Should we invest in Tesla given the current stiuation of EV")
]})

Plan(steps=['Research the current state of the EV market (market share, growth rate, and major players).', "Analyze Tesla's current market position, financial performance, and competitive advantages.", 'Evaluate potential risks and opportunities for Tesla in the EV market.', 'Provide a recommendation based on the analysis.'])

In [20]:
from  typing import Union

class Resposne(BaseModel):
    """Resposne To the user """
    
    response:str

class Act(BaseModel):
    """Action to be performed """

    action: Union[Resposne, Plan] = Field(
        description="ACtion to perform. Yf you wnat to respond to user, use Resposne."
        "If you further want to use tools to get the answer use Plan"
    )


replanner_promt=ChatPromptTemplate.from_template(
    """For the gven objective come up with a structured output in simple step by step
     Your object was this :
     {input}

     Your original plan was this:
     {plan}

     you have currently done the follow steps:
     {past_steps}


     Update your plan accordingly 
    """
)

replaner = replanner_promt|llm.with_structured_output(Act)

replaner.invoke({
    'input': "Should we invest in Tesla given the current situation of EV?",
    'plan': "Step 1: Analyze Tesla’s Q1 earnings. Step 2: Review EV market trends. Step 3: Check competitor growth.",
    'past_steps': "Completed Step 1: Tesla’s Q1 earnings analyzed."
})


Act(action=Plan(steps=['Review EV market trends.', 'Check competitor growth.', 'Make investment decision based on analysis.']))